In [0]:
a = lambda x: x*x
print(a(5))

In [0]:
import pandas as pd

df = pd.DataFrame([[1001, 'John', 20], 
                   [1002, 'Kunal', 23],
                   [1003, 'Sonali', 25],
                   [1004, 'Madhabi', 21]], 
                  columns=['ID', 'Name', 'Age'])

display(df)

In [0]:
pip install openpyxl

In [0]:
%restart_python

In [0]:
import pandas as pd
# import openpyxl

df_books = pd.read_csv('/Volumes/workspace/databricks_work/all_datafiles/books_dataset.csv')
display(df_books)


In [0]:
condition  = ~df_books['isbn'].str.contains('-')
df_books.loc[condition,'isbn']=df_books.loc[condition,'isbn'].str.slice(0,3)+'-'+df_books.loc[condition,'isbn'].str.slice(3)

display(df_books)

In [0]:
df_to_10_page_books = df_books.nlargest(10,'pages')
display(df_to_10_page_books)

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark Dataframe").getOrCreate()
books_s_df = spark.createDataFrame(df_books)
display(books_s_df)

In [0]:
from pyspark.sql.functions import col, when, concat, lit

books_s_df = books_s_df.withColumn(
    'isbn', 
    when(
        # Parentheses around the first condition AND the second condition
        (col('isbn').isNotNull()) & (col('isbn').substr(8, 1) != '-'), 
        concat(col('isbn').substr(1, 7), lit('-'), col('isbn').substr(8, 100))
    ).otherwise(col('isbn'))
)

display(books_s_df)

In [0]:
for name in books_s_df.columns:
    books_s_df = books_s_df.withColumnRenamed(name, name.strip())
books_s_df.write.mode("overwrite").saveAsTable('books_basic_info')